In [5]:
import numpy as np
import os

from pathlib import Path

# change working directory to make src visible
os.chdir(Path.cwd().parent)

from src.data_generation import SPDEventGenerator
from src.visualization import draw_event

In [6]:
event_gen = SPDEventGenerator(detector_eff=0.98)

for _ in range(2):
    event = event_gen.generate_spd_event()
    print(event)

Event:
Shape of real hits: (207, 3)
Shape of momentums: (207, 3)
Shape of fake hits: (962, 3)
Fraction of fakes: 0.82
Fraction of missing hits: 0.01
Number of unique tracks: 6
Vertex: Vertex(x=-4.58, y=5.05, z=-119.62)
Track parameters:
	Track ID: 0, TrackParams(pt=180.19, phi=3.77, theta=0.44, charge=1)
	Track ID: 1, TrackParams(pt=348.15, phi=4.64, theta=1.48, charge=-1)
	Track ID: 2, TrackParams(pt=170.50, phi=2.47, theta=1.01, charge=-1)
	Track ID: 3, TrackParams(pt=399.25, phi=4.11, theta=0.83, charge=-1)
	Track ID: 4, TrackParams(pt=740.25, phi=2.56, theta=1.55, charge=-1)
	Track ID: 5, TrackParams(pt=915.80, phi=5.16, theta=2.44, charge=1)


Event:
Shape of real hits: (173, 3)
Shape of momentums: (173, 3)
Shape of fake hits: (668, 3)
Fraction of fakes: 0.79
Fraction of missing hits: 0.01
Number of unique tracks: 5
Vertex: Vertex(x=-8.08, y=-13.58, z=-64.11)
Track parameters:
	Track ID: 0, TrackParams(pt=744.25, phi=0.59, theta=1.43, charge=1)
	Track ID: 1, TrackParams(pt=220.01,

In [7]:
draw_event(
    hits=event.hits,
    fakes=event.fakes,
    vertex=event.vertex.numpy,
    labels=event.track_ids,
)

In [8]:
from src.normalization import ConstraintsNormalizer

normalizer = ConstraintsNormalizer()

draw_event(
    hits=normalizer(event.hits),
    fakes=normalizer(event.fakes),
    vertex=normalizer(np.expand_dims(event.vertex.numpy, 0)).squeeze(0),
    labels=event.track_ids,
    x_coord_range=[-1, 1],
    y_coord_range=[-1, 1],
    z_coord_range=[-1, 1],
)